In [1]:
import pandas as pd
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDRegressor, Lars, BayesianRidge, LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import pystacknet
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn 

from scipy import stats
from scipy.stats import norm, skew, boxcox
from scipy.special import boxcox1p, inv_boxcox
def metric(y_true, y_pred, sample_weight=None):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [2]:
fnc = pd.read_csv('fnc.csv')
loading = pd.read_csv('loading.csv')
reveal = pd.read_csv('reveal_ID_site2.csv')
sample = pd.read_csv('sample_submission.csv')
train_scores = pd.read_csv('train_scores.csv')

fnc_features, loading_features = list(fnc.columns[1:]), list(loading.columns[1:])
loading = loading.drop(['IC_20'], axis=1)
loading_features.remove('IC_20')

In [3]:
df = fnc.merge(loading, on="Id")

In [4]:
np.random.seed(0)
SEED = 0
NUM_FOLDS = 7
FNC_SCALE = 1/500

In [5]:
scaler = MinMaxScaler()
df.iloc[:,1:] = scaler.fit_transform(df.iloc[:,1:])
scaler = RobustScaler()
df.iloc[:,1:] = scaler.fit_transform(df.iloc[:,1:])

train_scores["is_train"] = True
df = df.merge(train_scores, on="Id", how="left")

test_df = df[df["is_train"] != True].copy()
df = df[df["is_train"] == True].copy()

In [6]:
df[fnc_features] *= FNC_SCALE
test_df[fnc_features] *= FNC_SCALE

In [7]:
df.age += np.random.uniform(-1.0, 1.0, df.shape[0])
df.domain1_var1 += np.random.uniform(-1.0, 1.0, df.shape[0])
df.domain1_var2 += np.random.uniform(-1.0, 1.0, df.shape[0])
df.domain2_var1 += np.random.uniform(-1.0, 1.0, df.shape[0])
df.domain2_var2 += np.random.uniform(-1.0, 1.0, df.shape[0])

In [8]:
ridge = Ridge(alpha=0.004)
lasso = Lasso(alpha=0.00009)
enet = ElasticNet(alpha=0.000003, l1_ratio=0.75)
sgd = SGDRegressor(alpha=0.0003, l1_ratio=0.75 )
lars = Lars(n_nonzero_coefs = 150)
b_ridge = BayesianRidge(n_iter=3000)
lr = LinearRegression()

models = [ [ridge, lasso, enet, sgd, lars, b_ridge ], 
           [lr] ]

In [9]:
target = 'age'

from pystacknet.pystacknet import StackNetRegressor

features = loading_features + fnc_features
model = StackNetRegressor( models, 
                           metric=metric, 
                           folds=7,
                           restacking=False,
                           use_retraining=True,
                           random_state=SEED,
                           n_jobs=-1, 
                           verbose=1)

# Fit the entire model tree
model.fit(df[df[target].notnull()][features], df[df[target].notnull()][target])

====================== Start of Level 0 ======================
Input Dimensionality 1403 at Level 0 
6 models included in Level 0 
Fold 1/7 , model 0 , custom===0.143157 
Fold 1/7 , model 1 , custom===0.142284 
Fold 1/7 , model 2 , custom===0.143144 
Fold 1/7 , model 3 , custom===0.153297 
Fold 1/7 , model 4 , custom===0.148084 
Fold 1/7 , model 5 , custom===0.153136 
=========== end of fold 1 in level 0 ===========
Fold 2/7 , model 0 , custom===0.141160 
Fold 2/7 , model 1 , custom===0.141110 
Fold 2/7 , model 2 , custom===0.141147 
Fold 2/7 , model 3 , custom===0.152896 
Fold 2/7 , model 4 , custom===0.146081 
Fold 2/7 , model 5 , custom===0.152425 
=========== end of fold 2 in level 0 ===========
Fold 3/7 , model 0 , custom===0.141752 
Fold 3/7 , model 1 , custom===0.143222 
Fold 3/7 , model 2 , custom===0.141769 
Fold 3/7 , model 3 , custom===0.150535 
Fold 3/7 , model 4 , custom===0.148380 
Fold 3/7 , model 5 , custom===0.150522 
=========== end of fold 3 in level 0 ===========
Fo

In [41]:
#does not cater to the needs, not helpful :(